We built a model in 3.0-theberling-classifier.ipynb but we didn't have the right submissions. So now we're going to reverse engineer from their sample submission file to get the year, and team IDs that we need. Let's start by reading in their sample submission data file.

In [1]:
import pandas as pd

In [2]:
df_submission_sample = pd.read_csv('../../data/raw/MDataFiles_Stage2/MSampleSubmissionStage2.csv')

In [3]:
df_submission_sample.head()

,ID,Pred
0,2021_1101_1104,0.5
1,2021_1101_1111,0.5
2,2021_1101_1116,0.5
3,2021_1101_1124,0.5
4,2021_1101_1140,0.5


In [4]:
new = df_submission_sample["ID"].str.split("_", expand = True)
new

,0,1,2
0,2021,1101,1104
1,2021,1101,1111
2,2021,1101,1116
3,2021,1101,1124
4,2021,1101,1140
...,...,...,...
2273,2021,1452,1457
2274,2021,1452,1458
2275,2021,1455,1457
2276,2021,1455,1458


In [5]:
combinations = pd.DataFrame()
combinations['Season'] = new[0]
combinations['LowID'] = new[1]
combinations['HighID'] = new[2]
combinations['Season'] = pd.to_numeric(combinations['Season'])
combinations['LowID'] = pd.to_numeric(combinations['LowID'])
combinations['HighID'] = pd.to_numeric(combinations['HighID'])
combinations.head()

,Season,LowID,HighID
0,2021,1101,1104
1,2021,1101,1111
2,2021,1101,1116
3,2021,1101,1124
4,2021,1101,1140


Now let's load up the season averages data.

In [9]:
df = pd.read_csv('../../data/processed/data_averages_2021.csv')

Let's merge the low ID team with it's data by season. To do that we'll temporarily rename LowID to TeamID and merge on Season and TeamID.

In [10]:
LowID = pd.merge(combinations.rename({'LowID': 'TeamID'}, axis=1), df, how="left", on=['Season','TeamID']).dropna().reset_index(drop=True)
LowID.head()

,Season,TeamID,HighID,Score,FGP,FGP3,FTP,OR,DR,Ast,TO,Stl,Blk,PF,Rank,Seed
0,2021,1101,1104,76.304348,0.459648,0.383192,0.689699,9.608696,22.826087,18.173913,13.434783,9.347826,3.0,5.73913,83.48,14.0
1,2021,1101,1111,76.304348,0.459648,0.383192,0.689699,9.608696,22.826087,18.173913,13.434783,9.347826,3.0,5.73913,83.48,14.0
2,2021,1101,1116,76.304348,0.459648,0.383192,0.689699,9.608696,22.826087,18.173913,13.434783,9.347826,3.0,5.73913,83.48,14.0
3,2021,1101,1124,76.304348,0.459648,0.383192,0.689699,9.608696,22.826087,18.173913,13.434783,9.347826,3.0,5.73913,83.48,14.0
4,2021,1101,1140,76.304348,0.459648,0.383192,0.689699,9.608696,22.826087,18.173913,13.434783,9.347826,3.0,5.73913,83.48,14.0


In [11]:
LowID.columns = ['Season','LowID','HighID','LowScore','LowFGP','LowFGP3','LowFTP','LowOR','LowDR','LowAst','LowTO','LowStl','LowBlk','LowPF','LowRank','LowSeed']
LowID.head()

,Season,LowID,HighID,LowScore,LowFGP,LowFGP3,LowFTP,LowOR,LowDR,LowAst,LowTO,LowStl,LowBlk,LowPF,LowRank,LowSeed
0,2021,1101,1104,76.304348,0.459648,0.383192,0.689699,9.608696,22.826087,18.173913,13.434783,9.347826,3.0,5.73913,83.48,14.0
1,2021,1101,1111,76.304348,0.459648,0.383192,0.689699,9.608696,22.826087,18.173913,13.434783,9.347826,3.0,5.73913,83.48,14.0
2,2021,1101,1116,76.304348,0.459648,0.383192,0.689699,9.608696,22.826087,18.173913,13.434783,9.347826,3.0,5.73913,83.48,14.0
3,2021,1101,1124,76.304348,0.459648,0.383192,0.689699,9.608696,22.826087,18.173913,13.434783,9.347826,3.0,5.73913,83.48,14.0
4,2021,1101,1140,76.304348,0.459648,0.383192,0.689699,9.608696,22.826087,18.173913,13.434783,9.347826,3.0,5.73913,83.48,14.0


In [12]:
HighID = pd.merge(combinations.rename({'HighID': 'TeamID'}, axis=1), df, how="left", on=['Season','TeamID']).dropna().reset_index(drop=True)
HighID.head()

,Season,LowID,TeamID,Score,FGP,FGP3,FTP,OR,DR,Ast,TO,Stl,Blk,PF,Rank,Seed
0,2021,1101,1104,79.566667,0.430893,0.348281,0.711088,10.400000,25.966667,14.133333,13.833333,8.633333,4.333333,4.933333,6.865385,2.0
1,2021,1101,1111,68.166667,0.405138,0.320992,0.699853,8.875000,22.041667,11.333333,11.125000,7.541667,3.166667,8.083333,180.360000,16.0
2,2021,1101,1116,82.392857,0.454653,0.331366,0.743755,10.571429,26.142857,14.928571,12.821429,8.035714,5.142857,4.035714,13.269231,3.0
3,2021,1101,1124,84.375000,0.493448,0.416049,0.703066,11.333333,22.375000,17.041667,12.166667,8.958333,3.750000,5.250000,3.038462,1.0
4,2021,1101,1140,77.560000,0.478139,0.364688,0.742710,8.320000,28.280000,16.160000,13.000000,4.680000,2.840000,4.600000,20.666667,6.0


In [13]:
HighID.columns = ['Season','LowID','HighID','HighScore','HighFGP','HighFGP3','HighFTP','HighOR','HighDR','HighAst','HighTO','HighStl','HighBlk','HighPF','HighRank','HighSeed']
HighID.head()

,Season,LowID,HighID,HighScore,HighFGP,HighFGP3,HighFTP,HighOR,HighDR,HighAst,HighTO,HighStl,HighBlk,HighPF,HighRank,HighSeed
0,2021,1101,1104,79.566667,0.430893,0.348281,0.711088,10.400000,25.966667,14.133333,13.833333,8.633333,4.333333,4.933333,6.865385,2.0
1,2021,1101,1111,68.166667,0.405138,0.320992,0.699853,8.875000,22.041667,11.333333,11.125000,7.541667,3.166667,8.083333,180.360000,16.0
2,2021,1101,1116,82.392857,0.454653,0.331366,0.743755,10.571429,26.142857,14.928571,12.821429,8.035714,5.142857,4.035714,13.269231,3.0
3,2021,1101,1124,84.375000,0.493448,0.416049,0.703066,11.333333,22.375000,17.041667,12.166667,8.958333,3.750000,5.250000,3.038462,1.0
4,2021,1101,1140,77.560000,0.478139,0.364688,0.742710,8.320000,28.280000,16.160000,13.000000,4.680000,2.840000,4.600000,20.666667,6.0


And finally we can merge the HighID and LowID DataFrame on Season, LowID, HighID, and Win since those should be the same in each DataFrame.

In [14]:
model_dataset = pd.merge(LowID, HighID, how='outer', on=['Season','LowID','HighID'])
model_dataset.head()

,Season,LowID,HighID,LowScore,LowFGP,LowFGP3,LowFTP,LowOR,LowDR,LowAst,...,HighFTP,HighOR,HighDR,HighAst,HighTO,HighStl,HighBlk,HighPF,HighRank,HighSeed
0,2021,1101,1104,76.304348,0.459648,0.383192,0.689699,9.608696,22.826087,18.173913,...,0.711088,10.400000,25.966667,14.133333,13.833333,8.633333,4.333333,4.933333,6.865385,2.0
1,2021,1101,1111,76.304348,0.459648,0.383192,0.689699,9.608696,22.826087,18.173913,...,0.699853,8.875000,22.041667,11.333333,11.125000,7.541667,3.166667,8.083333,180.360000,16.0
2,2021,1101,1116,76.304348,0.459648,0.383192,0.689699,9.608696,22.826087,18.173913,...,0.743755,10.571429,26.142857,14.928571,12.821429,8.035714,5.142857,4.035714,13.269231,3.0
3,2021,1101,1124,76.304348,0.459648,0.383192,0.689699,9.608696,22.826087,18.173913,...,0.703066,11.333333,22.375000,17.041667,12.166667,8.958333,3.750000,5.250000,3.038462,1.0
4,2021,1101,1140,76.304348,0.459648,0.383192,0.689699,9.608696,22.826087,18.173913,...,0.742710,8.320000,28.280000,16.160000,13.000000,4.680000,2.840000,4.600000,20.666667,6.0


In [15]:
model_dataset.shape

(2278, 29)

In [16]:
model_dataset.to_csv('../../data/processed/model_dataset_2021.csv',index=False)